# Data Visualisation

## Notes and Quick ideas

- Now do plots with the repetitions baked in. Show which hyperparameter combinations have the least variance

- Figure out what went wrong in the labelling
    - Potentially relabel validation/training/testing(!)
- Redo difference graph after the relabelling
- Give a description of the dataset, what's in it
- Load dataset onto zenodo
- Create a bridge from background chapter to how the models are used

### Meeting notes
- [x] Check (before writing results chapter) that the delay isn't too big
- [x] Make *very* sure that the model can be run in real time, with the gloves
- [x] Conf matrix should be %age of the class
- [x] Explain Conf matrix structure (diagonals/orientations/fingers) in thesis
- [x] Include 'dummy' models which perfectly predict only finger/orientation/hand, etc
     - put it in a separate section in methodology
- [x] Look into plotting error on FFNN
- Discuss precision/recall for 51 gesture FFNN/HMM/CuSUM  -> Why would this happen
- Error types: (wrong timestep) x (wrong gesture)
    - It seems like the FFNN is not getting the timestep wrong, it's just wrong
- Explore plots of hpars affecting regularization and validation performance
- Make note that the HMM is only predicting 200 g255 gestures

### Changes made

- F1 score was being set to NaN, resulting in the average being too high (and F1 ~= 1.0)
- Grid search was unable to explore the search space fully, so [Optuna](https://optuna.readthedocs.io/en/stable/) was used for the search.
    - Specifically, the [Tree-structured Parzen Estimator](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler) performs the search. Bergstra, James et al. “Algorithms for Hyper-Parameter Optimization.” NIPS (2011). [Explanatory Blog](http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html#tree-structured-parzen-estimators-tpe)
- 


# Data Preparation

In [ ]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

## Imports and setup

In [ ]:
# minimise me
%load_ext autoreload
%autoreload 2
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import matplotlib as mpl
import ipywidgets as widgets
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn
import tqdm
import logging as l
import tqdm
import yaml
import glob
from matplotlib.colors import LogNorm
import re
from sklearn.metrics import classification_report
from scipy.stats import f

mpl.rc('font', family='serif', serif='cmr10')
mpl.rc('axes.formatter', use_mathtext=True)


## Utility functions

In [ ]:
# minimise me
def prettify_col_name(x):
    return x.split('.')[-1].replace('_', ' ').title()

def calculate_prediction_ellipse(x, y, alpha=0.5):
    """Given some x and y data, calculate the (1-alpha) confidence ellipse."""
    data = np.column_stack((x, y)) # Combine x and y into a single data array
    num_dimensions = data.shape[1]
    num_data_points = data.shape[0]
    # Estimate the sample covariance matrix
    sample_covariance_matrix = np.cov(data, rowvar=False)
    # Calculate the sample mean for each dimension
    sample_mean = np.mean(data, axis=0)
    # Generate angles for the ellipse
    theta = np.linspace(0, 2*np.pi, num=100)
    # Calculate the radius of the ellipse. `f.ppf` is the inverse of the CDF
    radius = np.sqrt(
        num_dimensions * (num_data_points - 1) / (num_data_points - num_dimensions) *
        (1 + 1/num_data_points) * f.ppf(1 - alpha, num_dimensions, num_data_points - num_dimensions)
    )
#     print(sample_covariance_matrix)
    # Compute the Cholesky decomposition of the covariance matrix
    chol_cov_matrix = np.linalg.cholesky(sample_covariance_matrix)
    # Generate ellipse offset based on Cholesky decomposition
    ellipse_offset = np.outer(np.cos(theta), chol_cov_matrix[0, :]) + np.outer(np.sin(theta), chol_cov_matrix[1, :])
    # Calculate the points of the prediction interval ellipse
    prediction_ellipse_points = sample_mean + radius * ellipse_offset
    return prediction_ellipse_points

def get_npz_data_from_model(model_dir):
    """Given a directory of a model, return it's y_pred and y_true."""
    data = np.load(f'{model_dir}/y_val_true_y_val_pred.npz')
    y_true = data['y_true']
    y_pred = data['y_pred']
    return y_true, y_pred

def show_conf_mat_from_model(model_dir, ax=None):
    """Given a directory of a model, plot its confidence matrix"""
    y_true, y_pred = get_npz_data_from_model(model_dir)
    cm_val = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    p = vis.conf_mat(cm_val / cm_val.sum(axis=0), ax=ax)
    return p

## Load data

In [ ]:
# Read in data from hpar optimisation
paths = sorted(glob.glob('../saved_models/results_*_optuna.jsonl'))
dfs = map(
    lambda path: pd.read_json(path, lines=True),
    paths
)
# Concat the dataframes together, and then do a 
# copy to avoid a dataframe fragmentation warning
# Reset the index to avoid a seaborn error https://github.com/mwaskom/seaborn/issues/3291
df = pd.concat(dfs).reset_index(drop=True).copy()
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].fillna('51')
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].astype(int).astype(str)

df.groupby(['model_type', 'preprocessing.num_gesture_classes']).size()

## Calculate some auxillary values

In [ ]:
# Preprocess the data a little bit, and get a list of dependant variables
# Preprocess the df a bit to get some nice-to-use columns
df['ffnn.nodes_per_layer.1'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)
df['ffnn.nodes_per_layer.2'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None
)
df['ffnn.nodes_per_layer.3'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None
)
# Calculate ratios

avgs = ('macro avg', 'weighted avg')
metrics = ('f1-score', 'precision', 'recall')

for avg in avgs:
    for metric in metrics:
        df[f'ratio.{avg}.{metric}'] = df[f'trn.{avg}.{metric}'] / df[f'val.{avg}.{metric}']
        df[f'ratio.{avg}.{metric}'] = np.where(
            np.isfinite(df[f'ratio.{avg}.{metric}']),
            df[f'ratio.{avg}.{metric}'],
            np.nan
        )


# Print out a list of dependant variables
dep_vars = sorted([
    c for c in df.columns 
    if 'val' not in c and 'trn' not in c and 'ratio' not in c and c not in (
        'saved_at', 'fit_time', 'preprocessing.gesture_allowlist', 
)], key=lambda c: str(c))
print(f"Dependant variables: {dep_vars}")
# print("\nVariables which change:")
# max_len = max(map(lambda x: len(x), dep_vars))
# Print out all dependant variables that change
# for var in dep_vars:
#     uniq = df[var].apply(lambda x: str(x) if isinstance(x, list) else x).unique()
#     if len(uniq) > 1:
#         print(f"{var: <{max_len}} {uniq}")
        
df['ffnn.dropout_rate'] = np.round(df['ffnn.dropout_rate'], 6)

# Plotting

## Precision vs Recall vs $F_1$

### Precision vs Recall for all models

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
n_gesture_classes = ('51', '50', '5')

for ax, n_classes in zip(axs, n_gesture_classes):
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == n_classes],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=10,
        alpha=0.5,
        hue='model_type',
        ax=ax
    )
    ax.set_xlim((-0.1, 1.1))
    ax.set_ylim((-0.1, 1.1))
    ax.plot([0,1], [0,1], color='black', alpha=.1)
    ax.set_title(f'{n_classes} gesture classes')
plt.tight_layout()

### Precision vs Recall for FFNNs across hyperparameters

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
n_gesture_classes = ('51', '50', '5')

hyperpars = (
    'ffnn.dropout_rate.log10',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
    'ffnn.nodes_per_layer.1'
)
df[[
    'ffnn.dropout_rate.log10',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
]] = np.log10(df[[
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient',
    'nn.batch_size',
    'nn.learning_rate',
]])

fig, axs = plt.subplots(
    len(hyperpars), 
    len(n_gesture_classes), 
    figsize=(len(n_gesture_classes)*4, len(hyperpars)*4)
)
for i, hyperpar in enumerate(hyperpars):
    for j, n_classes in enumerate(n_gesture_classes):
        sns.scatterplot(
            data=df[
                (df['preprocessing.num_gesture_classes'] == n_classes) &
                (df['model_type'] == 'FFNN')
            ],
            x='val.macro avg.precision',
            y='val.macro avg.recall',
            s=10,
            hue=hyperpar,
            ax=axs[i, j]
        )
        axs[i, j].set_xlim((-0.1, 1.1))
        axs[i, j].set_ylim((-0.1, 1.1))
        axs[i, j].plot([0,1], [0,1], color='black', alpha=.1)
        axs[i, j].set_title(f'{n_classes} gesture classes')
plt.tight_layout()

### Distribution of validation $F_1$ scores

In [ ]:
rows = 'preprocessing.num_gesture_classes'
cols = 'model_type'
row_vars = df[rows].unique()
col_vars = df[cols].unique()
fig, axs = plt.subplots(
    len(row_vars),
    len(col_vars),
    figsize=(len(col_vars)*3, len(row_vars)*3),
    squeeze=False
)

for i, row in enumerate(row_vars):
    for j, col in enumerate(col_vars):
        data = df[
            (df[rows] == row) & 
            (df[cols] == col)
        ]
        sns.histplot(
            data=data,
            x='val.macro avg.f1-score',
            ax=axs[i,j],
            binwidth=.05
        )
        axs[i,j].set(
            title=f'{row} Gestures, {col}\n$F_1$ score',
            xlim=(0, 1)
        )
plt.tight_layout()

## Confusion Matrices of the best models

In [ ]:
ngestures = sorted(df['preprocessing.num_gesture_classes'].unique())
model_types = sorted(df['model_type'].unique())

fig, axs = plt.subplots(
    len(ngestures),
    len(model_types),
    figsize=(len(model_types)*6, len(ngestures)*6),
    squeeze=False
)

for i, ngesture in enumerate(ngestures):
    for j, model_type in enumerate(model_types):
        best = df[
            (df['preprocessing.num_gesture_classes'] == ngesture) &
            (df['model_type'] == model_type)
        ].sort_values('val.macro avg.f1-score', ascending=False)
        if len(best) == 0:
            continue
        best = best.iloc[0]
        print(ngesture, model_type, best['model_dir'])
        show_conf_mat_from_model(f"../{best['model_dir']}", axs[i, j])
        axs[i, j].set(
            title=f"Best {model_type}: {ngesture} gestures\n($F_1=${np.round(best['val.macro avg.f1-score'], 4)})",
        )

plt.tight_layout()

## Regularization Plots

In [ ]:
n_gesture_classes = (
    '5', 
    '50', 
    '51'
)
fig, axs = plt.subplots(len(n_gesture_classes), 2, figsize=(6, len(n_gesture_classes)*3))

for i, ngestures in enumerate(n_gesture_classes):
    data = df[df['preprocessing.num_gesture_classes'] == ngestures]
    sns.scatterplot(
        data=data,
        x='ffnn.l2_coefficient',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 0]
    )
    sns.scatterplot(
        data=data,
        x='ffnn.dropout_rate',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 1]
    )
    axs[i, 0].set_title(f'{ngestures} gestures')
    axs[i, 1].set_title(f'{ngestures} gestures')

plt.suptitle("$F_1$-ratio against regularisation")
plt.tight_layout()

## FFNN vs HMM vs CuSUM ($F_1$ scores)

In [ ]:
sns.catplot(
    data=df,
    x='model_type',
    y='val.macro avg.f1-score',
    col='preprocessing.num_gesture_classes',
    kind='violin',
)
plt.suptitle('$F_1$-score across different models and different #gestures')
plt.tight_layout()

In [ ]:
subset = df
(
    so.Plot(
        subset.sort_values(['preprocessing.num_gesture_classes', 'model_type']), 
        x='model_type', 
        y='val.macro avg.f1-score', 
        color='model_type'
    )
    .layout(size=(8, 6))
    .add(so.Dots(pointsize=3), so.Jitter())
    .facet(row='preprocessing.num_gesture_classes')
    .limit(y=(-0.05, 1.05))
    .label(
        x="Model Type",
        color='Model Type',
        y="Macro Average\n$F_1$ Score",
        title="{} Gesture Classes".format,
    )
)

## CuSUM Plots

In [ ]:
subset = df[df.model_type=='CuSUM'].sort_values('preprocessing.num_gesture_classes')
(
    so.Plot(subset, x='cusum.thresh', y='val.macro avg.f1-score')
    .layout(size=(8, 6))
    .add(so.Dots(pointsize=3), so.Jitter())
    .facet(col='preprocessing.num_gesture_classes')
    .limit(y=(-.1, 1.1))
    .label(
        x="CuSUM Threshold", 
#         color='Model Type',
#         y="Macro Average\n$F_1$ Score",
        title="CuSUM models\n({} Gesture Classes)".format,
    ).show()
)

sns.catplot(
    data=subset,
    x='cusum.thresh', 
    y='val.macro avg.f1-score',
    col='preprocessing.num_gesture_classes',
    kind='violin',
)

## HMM Plots

In [ ]:
subset = df[df.model_type=='HMM'].sort_values('preprocessing.num_gesture_classes')
(
    so.Plot(subset, x='preprocessing.num_gesture_classes', y='val.macro avg.f1-score')
#     .layout(size=(8, 6))
    .add(so.Dots(pointsize=3), so.Jitter())
    .limit(y=(-.1, 1.1))
    .label(
        x="Number of Gesture Classes", 
        y="Macro Average\n$F_1$ Score",
        title="HMM models",
    ).show()
)
sns.violinplot(
    data=subset,
    x='preprocessing.num_gesture_classes',
    y='val.macro avg.f1-score',
)

## FFNN Plots

### Pairplot of FFNN hyperparameters

In [ ]:
data = df[
    (df['model_type'] == 'FFNN') &
    (df['preprocessing.num_gesture_classes'] == '51')
]
hpars = (
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient',
    'nn.batch_size',
    'nn.learning_rate',
    'ffnn.nodes_per_layer.1'
)
metric = 'val.macro avg.f1-score'
hue = 'ratio.macro avg.f1-score'

rows = np.ceil(len(hpars) / 3).astype(int)
cols = min(3, len(hpars))
fig, axs = plt.subplots(
    rows,
    cols,
    figsize=(cols*6, rows*6)
)

for i, hpar in enumerate(hpars):
    sns.scatterplot(
        data=data,
        x=hpars[i],
        y=metric,
        ax=axs.flatten()[i],
        hue=hue
    )
    axs.flatten()[i].set_ylim((-0.05, 1.05))
    
plt.suptitle("Validation $F_1$ against FFNN hyperparameters")
plt.tight_layout()

### Heatmap-based pairplot of FFNN Hyperparameters

In [ ]:
data = df[
    (df['model_type'] == 'FFNN') &
    (df['preprocessing.num_gesture_classes'] == '51')
]

hpars_scale = [
    ('nn.learning_rate',       'log10'),
    ('ffnn.nodes_per_layer.1', 'log10'),
    ('ffnn.l2_coefficient',    'log10'),
    ('ffnn.dropout_rate',      'linear'),
]
# x_var_idx = 0
# y_var_idx = 1
def contour_nicely(x, y, z, xlabel, ylabel, xscale, yscale, fig, ax, levels=8):

    ax.tricontour(x, y, z, levels=levels, linewidths=0.25, colors='k')
    cntr2 = ax.tricontourf(x, y, z, levels=levels, cmap="RdBu_r")

    fig.colorbar(cntr2, ax=ax)
    ax.scatter(x, y, color='white', s=1)

    ax.set_xlabel(f'{xlabel} ({xscale})')
    if xscale == 'log10':
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels([f'{np.power(10, t):.3g}' for t in ax.get_xticks()])

    ax.set_ylabel(f'{ylabel} ({yscale})')
    if yscale == 'log10':
        ax.set_yticks(ax.get_yticks())
        ax.set_yticklabels([f'{np.power(10, t):.3g}' for t in ax.get_yticks()])

    ax.set_title(f'{xlabel} vs {ylabel}')

    
fig, axs = plt.subplots(
    len(hpars_scale), 
    len(hpars_scale), 
    dpi=200, 
    squeeze=False,
    figsize=(20,20)
)
z = data['val.macro avg.f1-score'].values
for x_var_idx in range(len(hpars_scale)):
    for y_var_idx in range(x_var_idx+1, len(hpars_scale)):
    
        x = data[hpars_scale[x_var_idx][0]].values
        if hpars_scale[x_var_idx][1] == 'log10':
            x = np.log10(x)
        elif hpars_scale[x_var_idx][1] != 'linear':
            raise NotImplementedError(f"Scale {hpars_scale[x_var_idx][0]} is not implemented")
        y = data[hpars_scale[y_var_idx][0]].values
        if hpars_scale[y_var_idx][1] == 'log10':
            y = np.log10(y)
        elif hpars_scale[y_var_idx][1] != 'linear':
            raise NotImplementedError(f"Scale {hpars_scale[y_var_idx][0]} is not implemented")
            
        contour_nicely(
            y, x, z,
            hpars_scale[y_var_idx][0],
            hpars_scale[x_var_idx][0],
            hpars_scale[y_var_idx][1],
            hpars_scale[x_var_idx][1],
            fig, axs[x_var_idx, y_var_idx]
        )
plt.tight_layout()
plt.show()



## Plot inference/training times

In [ ]:
sns.stripplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ].assign(**{
        'fit_time_per_obs': lambda x: x['fit_time'] / x['trn.num_observations']
    }),
    x='model_type',
    y='fit_time_per_obs'
)
plt.show()
sns.stripplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ].assign(**{
        'trn.pred_time_per_obs': lambda x: x['trn.pred_time'] / x['trn.num_observations']
    }),
    x='model_type',
    y='trn.pred_time_per_obs'
)
plt.show()
sns.stripplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ].assign(**{
        'val.pred_time_per_obs': lambda x: x['val.pred_time'] / x['val.num_observations']
    }),
    x='model_type',
    y='val.pred_time_per_obs'
)
plt.show()

## Example Confusion Matrices from baseline models

TODO: Also include models with perfect precision / perfect recall / perfect precision for non-g255 gestures / perfect precision for g255 / perfect recall for non-g255 gestures / perfect recall for g255

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

fig, axs = plt.subplots(2, 3, figsize=(16, 10))
axs = axs.flatten()

# Random model:
y_pred = np.random.randint(0, 51, y_trn.shape)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[0])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[0].set_title(f"Random model\n$F_1$={f1:.4}")

# Only predicts 50
y_pred = np.full(y_trn.shape, 50)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[1])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[1].set_title(f"Only predicts 50\n$F_1$={f1:.4}")

# Perfect, but random orientation:
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Add/subtract some random multiple of 10
    y_trn + 10*np.random.randint(
        -(y_trn // 10), 
        +(5 - y_trn // 10), 
        y_trn.shape
    ), 
    # Else do nothing
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[2])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[2].set_title(f"Perfect,\nbut random orientation\n$F_1$={f1:.4}")

# Perfect, but random finger:
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Then change only the last digit
    y_trn + np.random.randint(
        -np.mod(y_trn, 10), 
        +(10 - np.mod(y_trn, 10)),
        y_trn.shape
    ), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[3])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[3].set_title(f"Perfect,\nbut random finger\n$F_1$={f1:.4}")

# Perfect, but random finger (same hand):
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Then change only the last digit
    y_trn + np.random.randint(
        -np.mod(y_trn, 5), 
        +(5 - np.mod(y_trn, 5)),
        y_trn.shape
    ), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[4])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[4].set_title(f"Perfect,\nbut random finger on the correct hand\n$F_1$={f1:.4}")

# Never predicts 50
y_pred = np.where(
    # If the gesture IS g255
    y_trn == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 50, y_trn.shape), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(
    y_trn, 
    y_pred,
).numpy()
vis.conf_mat(cm_val, ax=axs[5])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[5].set_title(f"Perfect,\nbut predicts 50 as a random gesture\n$F_1$={f1:.4}")

# plt.savefig('../../report/src/imgs/graphs/05_example_conf_mats.png')
plt.tight_layout()
plt.show()

## PCA decomposition only including the gesture classes

In [ ]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped[y_trn != 50])

argsort = np.argsort(y_trn[y_trn != 50])

hues = np.array([ "0°", "45°", "90°", "135°", "180°" ])
styles = np.array([ "L1", "L2", "L3", "L4", "L5", "R5", "R4", "R3", "R2", "R1" ])

fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)
sns.scatterplot(
    x=X_tfrm[:, 0][argsort],
    y=X_tfrm[:, 1][argsort],
    hue=hues[(y_trn[y_trn != 50][argsort] // 10)],
    style=styles[(y_trn[y_trn != 50][argsort] % 10)],
    s=10,
    ax=ax
)
plt.title("PCA plot of the training data\nExcluding gesture 50")
# TODO maybe have an overlay of the datapoints that the model gets wrong?
# TODO 3D plot would be cool, but it just kinda hangs

## PCA plots

### PCA decomposition including non-gesture class

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

In [ ]:
colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"])

fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)
mask = (y_trn == 50)
ax.scatter(
    X_tfrm[:, 0][mask],
    X_tfrm[:, 1][mask],
    color='black',
    alpha=0.1,
    s=5,
    edgecolor='none',
)
ax.scatter(
    X_tfrm[:, 0][~mask],
    X_tfrm[:, 1][~mask],
    color=colours[(y_trn[~mask] // 10)],
    alpha=0.75,
    s=5,
    edgecolor='none',
)
# plt.title("PCA plot of the training data\nExcluding gesture 50")

### PCA plot showing just an interesting subset

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

In [ ]:
colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"])

@interact(
    x_start=(-1500, 1500, 50),
    y_start=(-1500, 1500, 50),
    x_length=(-1500, 1500, 50),
    y_length=(-1500, 1500, 50),
)
def fn(x_start=-150, y_start=1000, x_length=500, y_length=500):
    x_finsh = x_start + x_length
    y_finsh = y_start + y_length

    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)

    selection_mask = (
        (x_start <= X_tfrm[:, 0]) & (X_tfrm[:, 0] <= x_finsh) &
        (y_start <= X_tfrm[:, 1]) & (X_tfrm[:, 1] <= y_finsh)
    )
    X_subset = X_tfrm[selection_mask]
    y_subset = y_trn[selection_mask]
#     ax.scatter(
#         X_subset[:, 0],
#         X_subset[:, 1],
#         c='black',
#         alpha=0.1,
#     )
    
    y_mask = (y_trn == 50)
    ax.scatter(
        X_subset[:, 0][y_subset == 50],
        X_subset[:, 1][y_subset == 50],
        color='black',
        alpha=0.1,
        s=20,
        edgecolor='none',
    )
    ax.scatter(
        X_subset[:, 0][y_subset != 50],
        X_subset[:, 1][y_subset != 50],
        color=colours[(y_subset[y_subset != 50] // 10)],
        alpha=0.75,
#         s=5,
        edgecolor='none',
    )


### PCA plot that connects sequential datapoints

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

order = np.argsort(dt_trn)
X_tfrm = X_tfrm[order]

In [ ]:


limit = 1000
@interact(start=(0, len(X_tfrm), 25), length=(0, len(X_tfrm), 50))
def fn(start=0, length=500):
    finsh = min(start + length, len(X_tfrm))
    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)
    ax.plot(
        X_tfrm[:, 0][start:finsh],
        X_tfrm[:, 1][start:finsh],
        zorder=0,
        c='black',
        alpha=0.1,
    )
    sns.scatterplot(
        x=X_tfrm[:, 0][start:finsh],
        y=X_tfrm[:, 1][start:finsh],
        hue=dt_trn[order][start:finsh],
        legend=False,
        s=(10 + 90*(y_trn != 50)[order][start:finsh]),
        edgecolor=np.where((y_trn != 50)[order][start:finsh], 'black', 'none'),
        linewidth=.5,
    )
    
    idxs = np.nonzero(y_trn[order][start:finsh] != 50)[0]
    for idx in idxs:
        ax.text(
            X_tfrm[start:finsh][idx, 0],
            X_tfrm[start:finsh][idx, 1],
            y_trn[order][start:finsh][idx],
            va='center',
            ha='center',
        )
#     ax.set_xlim((
#         X_tfrm[:, 0].min() / 1.1,
#         X_tfrm[:, 0].max() * 1.1,
#     ))
#     ax.set_ylim((
#         X_tfrm[:, 1].min() / 1.1,
#         X_tfrm[:, 1].max() * 1.1,
#     ))

### PCA plot of a subset of the data

In [ ]:
subset = (
    (,),
    (,),
)
subset

## Visualise mis-predictions

1. Load in a continuous dataset
2. Load in a classifier
3. Use the classifier to make predictions on the dataset
4. Visualise the mispredictions, but *with context*

### Load in a model for which to evaluate the mis-predictions

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

clf = models.load_tf('../src/saved_models/ffnn_2023-09-18T14:05:16.363404')
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

### Visualise True and Mispredicted gestures

Plot all the observations which have the ground truth being gesture 255 but the model is not predicting g255

In [ ]:
y_pred = clf.predict(X_val)

for gidx in np.unique(y_val):
    if gidx == 50: continue
    pred_indxs = np.nonzero((y_val == 50) & (y_pred == gidx))[0]
    true_indxs = np.nonzero(y_val == gidx)[0]
    axs = vis.cmp_ts(
        X_val[true_indxs],
    )
    vis.cmp_ts(
        X_val[pred_indxs],
        color='tab:red',
        axs=axs,
    )

#     distances = np.abs(true_indxs[:, np.newaxis] - pred_indxs).min(axis=0)

    plt.suptitle(f'Model predicted {gidx}, ground truth: 50 \
                 \nGesture {gidx} in grey, mispredicted in red ({len(pred_indxs)} observations) \
                 \nindices: {pred_indxs}')
    plt.tight_layout()
#     plt.savefig(f'../src/notebooks/pred_{gidx:0>2}_truth_50.png', bbox_inches='tight')
    plt.show()
#     if gidx > 5:
    break


# Interactive plot to see data at a certain time

In [ ]:

df = read.read_data(
    '../gesture_data/train/', 
#     constants_path='../src/constants.yaml',
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)

@interact(dt='2022-10-08T20:23:46.665276000')
def fn(dt='2022-10-08T20:23:46.665276000'):
    try:
        dt = pd.to_datetime(dt)
    except Exception as e:
        print(e)
        return
#     fig, ax = plt.subplots(1, 1, figsize=(20, 6))
    mask = df['datetime'].between(
        dt - pd.to_timedelta(1, 'second'),
        dt + pd.to_timedelta(1, 'second')
    )
    
    vis.cmp_ts(
        [df.loc[mask, sensor_names].values]
    )
#     ax.plot(
#         df.loc[mask, sensor_names].values
#     )
#     dt_labels = df.loc[mask, 'datetime']
#     gidx_labels = df.loc[mask, 'gidx']
#     ax.set_xticks(range(len(dt_labels)))
#     ax.set_xticklabels([
#         f'{gidx_label} {str(dt_label)[5:-3]}'
#         for dt_label, gidx_label
#         in zip(dt_labels, gidx_labels)
#     ], rotation=90)
# 2022-10-08T20:23:46.665276000

# Misc Research Chapter Plots

### Read in all the data

In [ ]:
df = read.read_data(
    '../gesture_data/train/', 
    constants_path='../src/constants.yaml'
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

## Time-series heatmap + line plots

In [ ]:

def plt_subset(s, f):
    df = read.read_data(
        '../gesture_data/train/', 
        constants_path='../src/constants.yaml'
    )
    df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
    const = common.read_constants('../src/constants.yaml')
    sensor_names = list(const['sensors'].values())
    data = df[sensor_names].values[s:f]
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 6))
    sns.heatmap(
        data.T,
        ax=axs[0],
        cbar=False
    )

    axs[1].plot(
        data,
        color='black',
        alpha=0.2,
        linewidth=1,
    )
    plt.margins(0)
    plt.show()
# plt_subset(91_000, 95_000)
plt_subset(93_000, 93_200)

## Histogram of class distributions

In [ ]:
df['gidx'].hist()
plt.show()
df.loc[df['gidx'] != 50, 'gidx'].hist()
df['gidx'].value_counts() / len(df['gidx']) * 100

## All observations of one gesture

In [ ]:
gidx = 0
before = 10
after = 10
idxs = np.nonzero(df['gidx'] == gidx)[0][:, np.newaxis] + np.arange(-before, after+1)

vis.cmp_ts(df[sensor_names].values[idxs + 10]);
plt.tight_layout()
plt.show()

In [ ]:
np.arange(-before, after+1)

## 3D plot of the raw acceleration data

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
vals = df[['l5x', 'l5y', 'l5z']].values[:10000]
ax.plot(
    vals[:, 0], 
    vals[:, 1], 
    vals[:, 2], 
    label='3D Line'
)